<a href="https://colab.research.google.com/github/ny-yo/kaggle-CommonLit-Readability-Prize/blob/main/sample_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
# download API key from google drive
## Original: https://colab.research.google.com/drive/1eufc8aNCdjHbrBhuy7M7X6BGyzAyRbrF#scrollTo=y5_288BYp6H1
## When you run for the first time, you will see a link to authenticate.

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [3]:
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes        188           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes        321           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        936           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       5876           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      48320            True  
house-prices-advanced-regression-techniques    2030-01-01 00:00:00  Getting Started  Knowledge      12614            True  
connectx

In [4]:
!kaggle competitions download -c titanic

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.90MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.6MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 52.0MB/s]


In [12]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import time
from tensorflow.python.framework import ops
ops.reset_default_graph()
def parse(df):
    # Nan を -1 に置換
    df["Age"] = df["Age"].fillna(-1)
    df["Embarked"] = df["Embarked"].fillna(-1)
    # 文字列を数値に変換
    df["Sex"][df["Sex"] == "male"] = 0
    df["Sex"][df["Sex"] == "female"] = 1
    df["Embarked"][df["Embarked"] == "S" ] = 0
    df["Embarked"][df["Embarked"] == "C" ] = 1
    df["Embarked"][df["Embarked"] == "Q"] = 2
    return df

def split_val(x,y,rate,seed=None):
    # x,y の rate[%] をランダム分割
    N = x.shape[0]
    val_num = int(N*rate)
    if seed is not None:
        np.random.seed(seed)
    perm = np.random.permutation(N)
    ti = perm[:-val_num]
    vi = perm[-val_num:]
    return x[ti],y[ti],x[vi],y[vi]

def load_data():
    train_csv = pd.read_csv('train.csv')
    train_csv = parse(train_csv)
    train_x = train_csv[['Pclass', 'Sex', 'Fare','SibSp', 'Parch', 'Age', 'Embarked']].values
    train_y = train_csv['Survived'].values
    return split_val(train_x,train_y,0.1)

Instructions for updating:
non-resource variables are not supported in the long term


In [13]:
def create_model(input_placeholder,u_dim,layer_num,y_dim,training):
    h = input_placeholder
    for i in range(layer_num):
        h = tf.layers.dense(inputs=h, units=u_dim,activation=tf.nn.relu)
        h = tf.layers.batch_normalization(h,training=training)  
    h = tf.layers.dense(inputs=h, units=y_dim)
    return h

In [14]:
np.random.seed(0)
tf.reset_default_graph()
y_dim     = 2
# hyperparameters
u_dim     = 100
layer_num = 2
epoch     = 1000
log_freq  = 100
batchsize = 200
lr        = 0.001

# data load
train_x,train_y,val_x,val_y = load_data()
N  = train_y.shape[0]
Nv = val_y.shape[0]
print('train num: {}, val num: {}'.format(N,Nv))

# calculation graph
x = tf.placeholder(tf.float32, [None, train_x.shape[1]],"input")
y = tf.placeholder(tf.int32, [None])

with tf.variable_scope("model"):
  train_z = create_model(x,u_dim,layer_num,y_dim,training=True)
with tf.variable_scope("model", reuse=True):
  z       = create_model(x,u_dim,layer_num,y_dim,training=False)

cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=train_z)

extra_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(extra_ops):
  train_step = tf.train.AdamOptimizer(lr).minimize(cross_entropy)

pred_y = tf.cast(tf.argmax(z, 1), tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred_y, y), tf.float32))

# main loop
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
T = time.time() 
for ep in range(1,epoch+1) :
  perm=np.random.permutation(N)
  for i in range(0,N,batchsize):
    batch_xs=train_x[perm[i:i+batchsize]]
    batch_ys=train_y[perm[i:i+batchsize]]
    sess.run(train_step, feed_dict={x: batch_xs, y: batch_ys})
  # monitor
  if ep%log_freq ==0 :      
    train_loss, train_acc = sess.run([cross_entropy, accuracy], feed_dict={x: train_x, y: train_y})
    val_loss, val_acc     = sess.run([cross_entropy, accuracy], feed_dict={x: val_x, y: val_y})
    epochT = time.time()-T
    print('Epoch: %d, Time :%.4f (s), train_loss: %f,  train_acc: %f, val_loss: %f,  val_acc: %f' % (ep, epochT, train_loss, train_acc, val_loss, val_acc))
    T = time.time()

train num: 802, val num: 89
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_l

Epoch: 100, Time :2.3009 (s), train_loss: 0.418984,  train_acc: 0.718204, val_loss: 0.401131,  val_acc: 0.786517
Epoch: 200, Time :0.9462 (s), train_loss: 0.437863,  train_acc: 0.764339, val_loss: 0.583735,  val_acc: 0.685393
Epoch: 300, Time :0.9648 (s), train_loss: 0.410202,  train_acc: 0.805486, val_loss: 0.486046,  val_acc: 0.775281
Epoch: 400, Time :0.9427 (s), train_loss: 0.402765,  train_acc: 0.827930, val_loss: 0.471878,  val_acc: 0.808989
Epoch: 500, Time :0.9569 (s), train_loss: 0.388416,  train_acc: 0.824190, val_loss: 0.490891,  val_acc: 0.786517
Epoch: 600, Time :0.9510 (s), train_loss: 0.386889,  train_acc: 0.821696, val_loss: 0.417588,  val_acc: 0.831461
Epoch: 700, Time :0.9593 (s), train_loss: 0.385314,  train_acc: 0.806733, val_loss: 0.435373,  val_acc: 0.831461
Epoch: 800, Time :0.9590 (s), train_loss: 0.393612,  train_acc: 0.812968, val_loss: 0.471411,  val_acc: 0.775281
Epoch: 900, Time :0.9488 (s), train_loss: 0.382463,  train_acc: 0.840399, val_loss: 0.465884,  v